In [1]:
import spacy
from evaluator import test_topics, test_qrels, train_topics, train_qrels
from query_model import POSQuery
import elasticsearch
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
import os
import evaluator
load_dotenv()

True

In [2]:
es = Elasticsearch(http_auth=(os.environ['ES_USER'], os.environ['ES_PWD']))

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
qm = POSQuery(es, 'trec2019_stem', nlp)

In [5]:
%%time
qm.fit(train_topics, train_qrels, 100)

Done computing perfect queries for category 1
Done computed perfect queries for category 2
CPU times: user 27.5 s, sys: 490 ms, total: 28 s
Wall time: 6min 50s


In [6]:
data, scores, turn_depth_scores, qdata = evaluator.evaluate(es,
                                                            'trec2019_stem',
                                                            size=1000,
                                                            qm=qm,
                                                            topics=test_topics,
                                                            qrels=test_qrels,
                                                            k=3)

In [7]:
ndcg_score = sum(scores)/len(scores)
ndcg_score

0.20142801194598395

In [8]:
FILENAME = 'ES_stemming_qm'

In [9]:
evaluator.to_run_file(data, FILENAME, 'Q0', 'es')

In [10]:
evaluator.to_run_file(data, FILENAME, 'Q0', 'es')

In [ ]:
evaluator.to_query_file(qdata, FILENAME)

# How does the model rewrite queries based on context?
Lets visualize it by running the first three questions within topic1 of the unseen test data

In [12]:
TOPIC_NUM = 10000

In [13]:
qm.get_query('What is throat cancer?', depth=1, topic_num=TOPIC_NUM)

['throat', 'cancer']

In [14]:
qm.get_query('Is it treatable?', depth=2, topic_num=TOPIC_NUM)

['treatable', 'throat', 'cancer']

In [15]:
qm.get_query('Tell me about lung cancer.', depth=3, topic_num=TOPIC_NUM)

['tell', 'lung', 'cancer', 'throat', 'treatable']